<h3 align = 'center'><font color='#28abe3'>Predicting Red Hat Business Value (Improving on Benchmark)</font></h3>

Update:

After doing a bit of reading of the kernals and forums for this contest there is a big leak in the data which alone produces an AUC of .987. Right now the 10% mark is an AUC of 0.991676 so I definitely need to exploit the leak in order to get into the top 10% in this competiton. The competition mods say there are around 70k test points unaffected by the leak (datapoints with a group_1 class not in the train set) so that last bit of improvement will come from true model improvements, feature engineering, ensembling etc. Lets start with the biggest model improvement first, which is integrating the leak into our model.


---

Import the data:

In [5]:
#import modules
import pandas as pd
import numpy as np
import os 
import sqlite3

#rename columns
act_train = pd.read_csv('act_train.csv')
act_test = pd.read_csv('act_test.csv')
peopleData = pd.read_csv('people.csv')

#lets just deal with the people data first.
for i in list(peopleData.columns):
    if i not in ['people_id','activity_id','date']:
        if peopleData[i].dtype == 'object':
            peopleData[i] = peopleData[i].fillna('type 0')
            peopleData[i] = peopleData[i].map(lambda x: x.split(' ')[1]).astype(np.int32)
        elif peopleData[i].dtype == 'bool' :
            peopleData[i] = peopleData[i].astype(np.int8)
            
#now lets do it for the act_train dataset
for i in list(act_train.columns):
    if i not in ['outcome','people_id','activity_id','date']:
        act_train[i] = act_train[i].fillna('type 0')
        act_train[i] = act_train[i].map(lambda x: x.split(' ')[1]).astype(np.int32)
    elif act_train[i].dtype == 'bool' :
        act_train[i] = act_train[i].astype(np.int8)
        
#and test
for i in list(act_test.columns):
    if i not in ['outcome','people_id','activity_id','date']:
        act_test[i] = act_test[i].fillna('type 0')
        act_test[i] = act_test[i].map(lambda x: x.split(' ')[1]).astype(np.int32)
    elif act_test[i].dtype == 'bool' :
        act_test[i] = act_test[i].astype(np.int8)
        
#join the dataframes
trainData = act_train.merge(peopleData,on = 'people_id')
testData = act_test.merge(peopleData,on = 'people_id')


So instead of building features etc i'm going to just fill in the test data that can be accurately filled in by the leak, and then only try and predict on the 70,000 remaining points of data using a model with no group_1 predictor since the remaining test data has no training data with the same group_1 classification.

In [59]:
testData['group_1'].unique() in trainData['group_1'].unique()
missingGroups = []
counter = 0
for i in testData['group_1'].unique():
    if i in trainData['group_1'].unique():
        counter = counter+1
    else:
        missingGroups.append(i)


/Users/Ben/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  if __name__ == '__main__':


In [60]:
testData2 = testData[testData['group_1'].isin(missingGroups)]

,people_id,activity_id,date_x,activity_category,char_1_x,char_2_x,char_3_x,char_4_x,char_5_x,char_6_x,...,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38
222,ppl_100179,act2_2277070,2022-09-02,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
223,ppl_100179,act2_3152044,2022-09-02,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
486,ppl_100322,act2_1008806,2023-01-20,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,70
487,ppl_100322,act2_2022187,2023-01-20,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,70
488,ppl_100322,act2_2182495,2023-02-18,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,70
489,ppl_100322,act2_2868576,2023-02-15,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,70
490,ppl_100322,act2_3947881,2022-09-27,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,70
491,ppl_100322,act2_4071983,2022-10-13,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,70
492,ppl_100322,act2_4556794,2023-02-16,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,70
493,ppl_100322,act2_471550,2023-01-11,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,70


[]